<a href="https://colab.research.google.com/github/senthilkumar-dimitra/NDVI-time-series-analysis/blob/expts/sugarcane_farm_veg_indices_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install geemap -q
!pip install pykalman -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.2 MB/s eta 0:00:00


In [9]:
import ee
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from scipy.signal import savgol_filter
import statsmodels.api as sm
from scipy.ndimage import gaussian_filter1d
from pykalman import KalmanFilter
import numpy as np
import matplotlib.pyplot as plt

# Authenticate and initialize the library
ee.Authenticate()
ee.Initialize(project='senthilkumar-dimitra')

In [2]:
# Define the coordinates for the AOI (single polygon)
coordinates = [(11.9339692, 9.6092636), (11.9378834, 9.607271), (11.937092, 9.590627), (11.9297, 9.591299)]  # Dangote sugarcane field co-ordinates in Nigeria

# Define the time range
start_date = '2019-01-01'
end_date = '2023-12-31'

# Function to mask clouds based on the QA60 band of Sentinel-2
def mask_clouds(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return image.updateMask(cloud_mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Function to calculate RECI
def calculate_reci(image):
    reci = image.expression(
        'B8 / B5 - 1', {
            'B8': image.select('B8'),  # NIR
            'B5': image.select('B5')   # Red-Edge
        }).rename('RECI')
    return image.addBands(reci)

# Function to calculate LSWI
def calculate_lswi(image):
    lswi = image.normalizedDifference(['B8', 'B11']).rename('LSWI')  # NIR and SWIR
    return image.addBands(lswi)

# Load Sentinel-2 data, filter by date and bounds, apply cloud mask, and calculate indices
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .map(mask_clouds) \
    .map(calculate_ndvi) \
    .map(calculate_reci) \
    .map(calculate_lswi)

# Define the polygon as an ee.Geometry
aoi = ee.Geometry.Polygon([coordinates])

# Calculate mean RECI, LSWI, and NDVI for the polygon over time
def get_index_timeseries(aoi, index_name):
    def compute_index(image):
        mean_index = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=30
        ).get(index_name)
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        return ee.Feature(None, {'date': date, index_name: mean_index})

    index_collection = collection.filterBounds(aoi).select([index_name]).map(compute_index)
    return index_collection.getInfo()

# Get the NDVI, RECI, and LSWI time series
ndvi_timeseries = get_index_timeseries(aoi, 'NDVI')
reci_timeseries = get_index_timeseries(aoi, 'RECI')
lswi_timeseries = get_index_timeseries(aoi, 'LSWI')

# Convert the time series to DataFrames
def extract_index_data(index_timeseries, index_name):
    dates = []
    index_values = []

    for feature in index_timeseries['features']:
        properties = feature['properties']
        date = properties.get('date')
        index_value = properties.get(index_name)
        if date and index_value is not None:
            dates.append(date)
            index_values.append(index_value)

    return pd.DataFrame({'Date': pd.to_datetime(dates), index_name: index_values})

# Create DataFrames for NDVI, RECI, and LSWI
ndvi_df = extract_index_data(ndvi_timeseries, 'NDVI')
reci_df = extract_index_data(reci_timeseries, 'RECI')
lswi_df = extract_index_data(lswi_timeseries, 'LSWI')

# Merge the DataFrames on the Date
merged_df = pd.merge(pd.merge(ndvi_df, reci_df, on='Date'), lswi_df, on='Date')

# Apply Savitzky-Golay filter to smooth the indices
merged_df['NDVI_Smoothed'] = savgol_filter(merged_df['NDVI'], window_length=11, polyorder=2)
merged_df['RECI_Smoothed'] = savgol_filter(merged_df['RECI'], window_length=11, polyorder=2)
merged_df['LSWI_Smoothed'] = savgol_filter(merged_df['LSWI'], window_length=11, polyorder=2)

In [3]:
# Fit linear regression models to detect trends for NDVI, RECI, and LSWI
merged_df['Date_ordinal'] = merged_df['Date'].map(pd.Timestamp.toordinal)

# NDVI trend
X_ndvi = sm.add_constant(merged_df['Date_ordinal'])
model_ndvi = sm.OLS(merged_df['NDVI_Smoothed'], X_ndvi).fit()
merged_df['NDVI_Trend'] = model_ndvi.predict(X_ndvi)

# RECI trend
X_reci = sm.add_constant(merged_df['Date_ordinal'])
model_reci = sm.OLS(merged_df['RECI_Smoothed'], X_reci).fit()
merged_df['RECI_Trend'] = model_reci.predict(X_reci)

# LSWI trend
X_lswi = sm.add_constant(merged_df['Date_ordinal'])
model_lswi = sm.OLS(merged_df['LSWI_Smoothed'], X_lswi).fit()
merged_df['LSWI_Trend'] = model_lswi.predict(X_lswi)

In [4]:
merged_df

,Date,NDVI,RECI,LSWI,NDVI_Smoothed,RECI_Smoothed,LSWI_Smoothed,Date_ordinal,NDVI_Trend,RECI_Trend,LSWI_Trend
0,2019-01-07,0.314894,0.513942,0.129045,0.416816,0.717166,0.101015,737066,0.415369,0.863115,0.059882
1,2019-01-12,0.449595,0.773928,0.081498,0.404893,0.690400,0.099316,737071,0.415420,0.863330,0.060353
2,2019-01-17,0.445153,0.795077,0.091194,0.388342,0.655531,0.092459,737076,0.415471,0.863545,0.060824
3,2019-01-22,0.418231,0.696943,0.053699,0.367163,0.612560,0.080444,737081,0.415522,0.863760,0.061295
4,2019-01-27,0.390681,0.657087,0.030061,0.341356,0.561485,0.063272,737086,0.415573,0.863976,0.061766
...,...,...,...,...,...,...,...,...,...,...,...
260,2023-12-07,0.662397,1.449089,0.196951,0.612561,1.358213,0.208211,738861,0.433679,0.940383,0.228963
261,2023-12-12,0.442554,0.858247,0.214610,0.620025,1.371731,0.203453,738866,0.433730,0.940599,0.229434
262,2023-12-17,0.630612,1.375803,0.192016,0.620169,1.363339,0.199510,738871,0.433781,0.940814,0.229905
263,2023-12-22,0.678495,1.553115,0.191308,0.612991,1.333037,0.196379,738876,0.433832,0.941029,0.230376


In [5]:
fig = go.Figure()

# Add original NDVI values and smoothed NDVI values
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['NDVI'], mode='markers+lines', name='NDVI (Original)'))
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['NDVI_Smoothed'], mode='markers+lines', name='NDVI (Smoothed)'))
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['NDVI_Trend'], mode='lines', name='NDVI Trend', line=dict(color='red')))

# Add RECI values and trends
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['RECI'], mode='markers+lines', name='RECI (Original)', line=dict(color='green')))
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['RECI_Smoothed'], mode='markers+lines', name='RECI (Smoothed)', line=dict(color='green')))
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['RECI_Trend'], mode='lines', name='RECI Trend', line=dict(color='darkgreen')))

# Add LSWI values and trends
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['LSWI'], mode='markers+lines', name='LSWI (Original)', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['LSWI_Smoothed'], mode='markers+lines', name='LSWI (Smoothed)', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['LSWI_Trend'], mode='lines', name='LSWI Trend', line=dict(color='darkblue')))

fig.update_layout(title='NDVI, RECI, and LSWI Trend Analysis (2019-2023)', xaxis_title='Date', yaxis_title='Index Value')
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [7]:
# Filter the DataFrame for a single year (e.g., 2020)
single_year_df = merged_df[(merged_df['Date'] >= '2020-01-01') & (merged_df['Date'] < '2021-01-01')]

fig = go.Figure()

# Add NDVI, RECI, and LSWI for the selected year
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['NDVI'], mode='markers+lines', name='NDVI'))
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['RECI'], mode='markers+lines', name='RECI', line=dict(color='green')))
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['LSWI'], mode='markers+lines', name='LSWI', line=dict(color='blue')))

fig.update_layout(title='NDVI, RECI, and LSWI for 2020', xaxis_title='Date', yaxis_title='Index Value',legend_title_text = 'Index')
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [10]:
# Ask the user to input the desired year
year = input("Enter the year for analysis (e.g., 2020, 2021): ")

# Convert the input year to an integer and filter the DataFrame
year = int(year)
single_year_df = merged_df[(merged_df['Date'] >= f'{year}-01-01') & (merged_df['Date'] < f'{year+1}-01-01')]

# Apply Savitzky-Golay filter
single_year_df['NDVI_SG'] = savgol_filter(single_year_df['NDVI'], window_length=7, polyorder=2)
single_year_df['RECI_SG'] = savgol_filter(single_year_df['RECI'], window_length=7, polyorder=2)
single_year_df['LSWI_SG'] = savgol_filter(single_year_df['LSWI'], window_length=7, polyorder=2)

# Apply Moving Average
single_year_df['NDVI_MA'] = single_year_df['NDVI'].rolling(window=5).mean()
single_year_df['RECI_MA'] = single_year_df['RECI'].rolling(window=5).mean()
single_year_df['LSWI_MA'] = single_year_df['LSWI'].rolling(window=5).mean()

# Apply Exponential Moving Average (EMA)
single_year_df['NDVI_EMA'] = single_year_df['NDVI'].ewm(span=5, adjust=False).mean()
single_year_df['RECI_EMA'] = single_year_df['RECI'].ewm(span=5, adjust=False).mean()
single_year_df['LSWI_EMA'] = single_year_df['LSWI'].ewm(span=5, adjust=False).mean()

# Apply Gaussian filter
single_year_df['NDVI_Gaussian'] = gaussian_filter1d(single_year_df['NDVI'], sigma=2)
single_year_df['RECI_Gaussian'] = gaussian_filter1d(single_year_df['RECI'], sigma=2)
single_year_df['LSWI_Gaussian'] = gaussian_filter1d(single_year_df['LSWI'], sigma=2)

# Apply LOESS (Locally Estimated Scatterplot Smoothing)
loess_ndvi = sm.nonparametric.lowess(single_year_df['NDVI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)
loess_reci = sm.nonparametric.lowess(single_year_df['RECI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)
loess_lswi = sm.nonparametric.lowess(single_year_df['LSWI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)

single_year_df['NDVI_LOESS'] = loess_ndvi[:, 1]
single_year_df['RECI_LOESS'] = loess_reci[:, 1]
single_year_df['LSWI_LOESS'] = loess_lswi[:, 1]

# Apply Kalman filter
kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)

single_year_df['NDVI_Kalman'] = kf.em(single_year_df['NDVI'], n_iter=5).smooth(single_year_df['NDVI'])[0]
single_year_df['RECI_Kalman'] = kf.em(single_year_df['RECI'], n_iter=5).smooth(single_year_df['RECI'])[0]
single_year_df['LSWI_Kalman'] = kf.em(single_year_df['LSWI'], n_iter=5).smooth(single_year_df['LSWI'])[0]


Enter the year for analysis (e.g., 2020, 2021): 2021


<ipython-input-10-9e7b22ed6065>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-9e7b22ed6065>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-9e7b22ed6065>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-9e7b2

In [11]:
fig = go.Figure()

# Add original NDVI, RECI, and LSWI values
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['NDVI'], mode='markers+lines', name='NDVI (Original)'))
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['RECI'], mode='markers+lines', name='RECI (Original)', line=dict(color='green')))
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['LSWI'], mode='markers+lines', name='LSWI (Original)', line=dict(color='blue')))

# Add smoothed NDVI, RECI, and LSWI values for each smoothing technique
smoothing_methods = ['SG', 'MA', 'EMA', 'Gaussian', 'LOESS', 'Kalman']
colors = {'SG': 'orange', 'MA': 'purple', 'EMA': 'cyan', 'Gaussian': 'pink', 'LOESS': 'brown', 'Kalman': 'red'}

for method in smoothing_methods:
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'NDVI_{method}'], mode='lines', name=f'NDVI ({method})', line=dict(color=colors[method])))
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'RECI_{method}'], mode='lines', name=f'RECI ({method})', line=dict(color=colors[method], dash='dash')))
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'LSWI_{method}'], mode='lines', name=f'LSWI ({method})', line=dict(color=colors[method], dash='dot')))

fig.update_layout(title=f'NDVI, RECI, and LSWI for {year} with Various Smoothing Techniques', xaxis_title='Date', yaxis_title='Index Value',legend_title_text = 'Index')
fig.show()


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

